In [1]:
#Libraries
import pandas as pd
import hashlib
import time
pd.set_option('display.max_rows', None)

In [2]:
# Function to fetch subject,object,property
def pre_process_data(df):
    
    df[0] = df[0].split(":")[1].strip()
    df[1] = df[1].split(":")[1].strip()
    df[2] = df[2].split(":")[1][:-1].strip() if len(df[2].split(":")) > 1 else df[2][:-1].strip()
    
    return df

In [3]:
# read 100k.txt into a pandas dataframe
raw_df = pd.read_csv("100k.txt", 
                     header = None, 
                     sep="\t", 
                     quotechar='"', 
                     skipinitialspace=True, 
                     names=['subject','property','object'])

raw_df = raw_df.apply(pre_process_data, axis=1)

In [4]:
# dict_df is a dictionary of dataframes with 
# key: property name, value: dataframe with subject, property and object for a given property
dict_df = dict(tuple(raw_df.groupby('property', as_index=False)))

# remove "property" column from every dataframe is dictionary dfs
for key in dict_df:
    dict_df[key] = dict_df[key].drop("property", axis=1)
    dict_df[key].reset_index(drop = True, inplace = True)

In [5]:
def sortMergeJoin(df1, key1, df2, key2):
    
    #Perform Sorting based on keys
    sort_df1_On_key1 = df1.sort_values(by=key1, ascending=True)
    sort_df2_On_key2 = df2.sort_values(by=key2, ascending=True)
    
    results_df = pd.merge(sort_df1_On_key1, sort_df2_On_key2, left_on=key1, right_on=key2, how='inner')

    return results_df    

In [6]:
# Sort Merge Join Results
start_time = time.time()

followsObj_friendOfSubj_smj_df = sortMergeJoin(
    dict_df["follows"],"object",
    dict_df["friendOf"],"subject")

likesObj_hasReviewSubj_smj_df = sortMergeJoin(
    dict_df["likes"],"object",
    dict_df["hasReview"],"subject")

friendsOfObj_likesSubj_smj_df = sortMergeJoin(
    followsObj_friendOfSubj_smj_df,"object_y",
    likesObj_hasReviewSubj_smj_df,"subject_x")

sortMergeJoin_QueryResult_df = friendsOfObj_likesSubj_smj_df[["subject_x_x",
                                     "object_x_x",
                                     "object_y_x",
                                     "object_x_y",
                                     "object_y_y"]]

sortMergeJoin_QueryResult_df.rename(columns = {'subject_x_x':'follows.subject',
                                       'object_x_x':'follows.object',
                                       'object_y_x':'friendOf.object',
                                       'object_x_y':'likes.object',
                                       'object_y_y':' hasReview.object'}, inplace = True)

end_time = time.time()

/tmp/ipykernel_4056/159088401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sortMergeJoin_QueryResult_df.rename(columns = {'subject_x_x':'follows.subject',


## Sort Merge Join Time Cost

In [7]:
print("Execution time for Sort merge Join: %s seconds" %(end_time - start_time))

Execution time for Sort merge Join: 7.9176695346832275 seconds


In [8]:
def hashJoin(df1, key1, df2, key2):
    
    df1["hash_key"] = df1[key1].apply(hash)
    df2["hash_key"] = df2[key2].apply(hash)
    
    df1_partitions = dict(tuple(df1.groupby('hash_key')))
    for key in df1_partitions:
        df1_partitions[key].reset_index(drop = True, inplace = True)
    
    dictKeys = df1_partitions.keys()
    resultAsList = []
    
    for i in range(len(df2)):
        if (df2.iloc[i]['hash_key']) in dictKeys:
            key = df2.iloc[i]['hash_key']
            joinRow = df2.iloc[[i]]
            joinedRow = pd.merge(df1_partitions[key],joinRow, on='hash_key',  how='left')
            resultAsList.append(joinedRow.to_dict('list'))
    
    listOfJoinedPartitions = []
    for dictionary in resultAsList:
        listOfJoinedPartitions.append(pd.DataFrame(dictionary))
    
    finalDf = pd.concat(listOfJoinedPartitions)
    finalDf = finalDf.drop('hash_key',axis=1)
    
    return finalDf

In [9]:
#Hash Join Results
start_time = time.time()

followsObj_friendOfSubj_hj_df = hashJoin(
    dict_df['follows'], 'object',
    dict_df['friendOf'], 'subject')

likesObj_hasReviewSubj_hj_df = hashJoin(
    dict_df['likes'], 'object',
    dict_df['hasReview'], 'subject')

friendsOfObj_likesSubj_hj_df = hashJoin(
    followsObj_friendOfSubj_hj_df, 'object_y',
    likesObj_hasReviewSubj_hj_df, 'subject_x')

hashJoin_QueryResult_df = friendsOfObj_likesSubj_hj_df[["subject_x_x",
                                     "object_x_x",
                                     "object_y_x",
                                     "object_x_y",
                                     "object_y_y"]]

hashJoin_QueryResult_df.rename(columns = {'subject_x_x': 'follows.subject',
                                          'object_x_x': 'follows.object',
                                          'object_y_x': 'friendOf.object',
                                          'object_x_y': 'likes.object',
                                          'object_y_y': 'hasReview.object'}, inplace = True)
                               
end_time = time.time()

/tmp/ipykernel_4056/325213909.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashJoin_QueryResult_df.rename(columns = {'subject_x_x': 'follows.subject',


## Hash Join Time Cost

In [10]:
print("Execution time for Hash Join: %s seconds" %(end_time - start_time))

Execution time for Hash Join: 196.18216705322266 seconds


In [11]:
def improvedHashJoin(df1, key1, df2, key2):
    
    df1["hash_key"] = df1[key1].apply(hash)
    df2["hash_key"] = df2[key2].apply(hash)
    
    df1_partitions = dict(tuple(df1.groupby('hash_key')))
    for key in df1_partitions:
        df1_partitions[key].reset_index(drop = True, inplace = True)

    df2_partitions = dict(tuple(df2.groupby('hash_key')))
    for key in df2_partitions:
        df2_partitions[key].reset_index(drop = True, inplace = True)
    
    resultAsList = []
    for key_2 in df2_partitions:
        for key_1 in df1_partitions:
            if key_1 == key_2:
                joinedPartitions = pd.merge(df1_partitions[key_1],df2_partitions[key_2], on='hash_key',  how='left')
                resultAsList.append(joinedPartitions.to_dict('list'))
    
    listOfJoinedPartitions = []
    for dictionary in resultAsList:
        listOfJoinedPartitions.append(pd.DataFrame(dictionary))
    
    finalDf = pd.concat(listOfJoinedPartitions)
    finalDf = finalDf.drop('hash_key',axis=1)
    return finalDf

In [12]:
#Improved Hash Join Results
start_time = time.time()

followsObj_friendOfSubj_ihj_df = improvedHashJoin(
    dict_df['follows'], 'object',
    dict_df['friendOf'], 'subject')

likesObj_hasReviewSubj_ihj_df = improvedHashJoin(
    dict_df['likes'], 'object',
    dict_df['hasReview'], 'subject')

friendsOfObj_likesSubj_ihj_df = improvedHashJoin(
    followsObj_friendOfSubj_ihj_df, 'object_y',
    likesObj_hasReviewSubj_ihj_df, 'subject_x')

improvedHashJoin_QueryResult_df = friendsOfObj_likesSubj_ihj_df[["subject_x_x",
                                     "object_x_x",
                                     "object_y_x",
                                     "object_x_y",
                                     "object_y_y"]]

improvedHashJoin_QueryResult_df.rename(columns = {'subject_x_x': 'follows.subject',
                                          'object_x_x': 'follows.object',
                                          'object_y_x': 'friendOf.object',
                                          'object_x_y': 'likes.object',
                                          'object_y_y': 'hasReview.object'}, inplace = True)
                               
end_time = time.time()

/tmp/ipykernel_4056/1294638882.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  improvedHashJoin_QueryResult_df.rename(columns = {'subject_x_x': 'follows.subject',


## Improved Hash Join Time Cost

In [13]:
print("Execution time for Improved Hash Join: %s seconds" %(end_time - start_time))

Execution time for Improved Hash Join: 27.514806509017944 seconds
